In [1]:
import nbimporter
import gas_water
import property_conversion  as prop

Importing Jupyter notebook from gas_water.ipynb
Importing Jupyter notebook from moisture_conversion.ipynb
Importing Jupyter notebook from vapour_pressure.ipynb
Importing Jupyter notebook from property_conversion.ipynb
Importing Jupyter notebook from liquid_water.ipynb


# 液水の状態を保持するモジュール

液水の状態を保持するクラスを作成する。  
液水の状態を表す方法として以下の3つの方法が挙げられる。  

- $temp$ 絶対温度, K
- $\phi$ 含水率, %  
- $\mu$ 水分化学ポテンシャル, J kg<sup>-1</sup>  
- $P_l$ 溶液の圧力, Pa  

多孔質材料中では吸着力によりある量の液水が保持される。この時、液水の保持量（含水率）と液水の状態量（水分化学ポテンシャル）は材料に応じた相関がある。（平衡含水率関係）  
しかしこの平衡関係は材料ごとに異なるため、平衡関係を用いる際に材料の情報を入力する必要がある。このモジュールでは、液水の状態を水分化学ポテンシャル（液相）を基準とし含水率を決定するクラスと、含水率を基準として水分化学ポテンシャル（液相）を決定するクラスの2つについて示す。

## クラス定義

### 1) 含水率ベース（気液平衡状態）

In [2]:
class PhiBasedWater():
    
    def __init__(self, temp, phi):
        self._temp = temp
        self._phi  = phi
        
    def set_material(self, material_name):
        self.material_name = material_name
    
    # temp
    
    def set_temp(self, temp):
        self._temp = temp
    
    def get_temp(self):
        return self._temp
    
    temp = property(get_temp, set_temp)

    # phi
    
    def set_phi(self, phi):
        self._phi = phi
        
    def get_phi(self):
        return self._phi
    
    phi = property(get_phi, set_phi)
    
    # miu
    
    def set_miu(self, miu):
        self._phi = prop.Kernel_Material_WaterAdsorption( self.material_name ).getPhi( MiuBasedWater(self.temp, miu) )
    
    def get_miu(self):
        return prop.Kernel_Material_WaterAdsorption( self.material_name ).getMiuByPhi( self )
    
    miu = property(get_miu, set_miu)
    
    # dphi
    
    def get_dphi(self):
        return prop.Kernel_Material_WaterAdsorption( self.material_name ).getDPhi( self )
    
    dphi = property(get_dphi)
    

### 2) 水分化学ポテンシャルベース（気液平衡状態）

In [3]:
class MiuBasedWater():
    
    def __init__(self, temp, miu):
        self._temp = temp
        self._miu  = miu
    
    def set_material(self, material_name):
        self.material_name = material_name
        
    # temp
    
    def set_temp(self, temp):
        self._temp = temp
    
    def get_temp(self):
        return self._temp
    
    temp = property(get_temp, set_temp)

    # miu
    
    def set_miu(self, miu):
        self._miu = miu
        
    def get_miu(self):
        return self._miu
    
    miu = property(get_miu, set_miu)
    
    # phi
    
    def set_phi(self, phi):
        self._miu = prop.Kernel_Material_WaterAdsorption( self.material_name ).getMiuByPhi( PhiBasedWater(self.temp, phi) )
    
    def get_phi(self):
        return prop.Kernel_Material_WaterAdsorption( self.material_name ).getPhi( self )
    
    phi = property(get_phi, set_phi)
    
    # dphi
    
    def get_dphi(self):
        return prop.Kernel_Material_WaterAdsorption( self.material_name ).getDPhi( self )
    
    dphi = property(get_dphi)
    

## 使い方

変数の宣言  
例として、
水蒸気の状態量を

- 絶対温度 300 K
- 相対湿度 0.9999(99.99%)  

とする。

In [4]:
vapour =  gas_water.RHBasedMoisture(temp=300.0, rh=0.9999)

材料の名前を'BentheimerSandstone'として気液平衡が成り立つとすると、  
液水のクラスは以下のように定義される。

In [5]:
water1 = MiuBasedWater( vapour.temp, vapour.miu)

In [6]:
water1.set_material('BentheimerSandstone')

水分化学ポテンシャルの取得

In [7]:
water1.miu

-13.848383372093412

含水率の取得

In [8]:
water1.phi

Importing Jupyter notebook from bentheimer_sandstone.ipynb
Importing Jupyter notebook from van_genuchten.ipynb


0.13286027466257927

含水率を与えて置き換え

In [9]:
water1.set_phi( 0.10 )

In [10]:
water1.miu

-20.29806887366382

同様に、含水率基準で考えると

In [11]:
water2 = PhiBasedWater( vapour.temp, 0.1)

In [12]:
water2.set_material('BentheimerSandstone')

含水率の取得

In [13]:
water2.phi

0.1

水分化学ポテンシャルを取得

In [14]:
water2.miu

-20.29806887366382

任意の水分化学ポテンシャルを与えて置き換え

In [15]:
water2.set_miu(-13.848383372093412)

In [16]:
water2.phi

0.13286027466257927

## Description

一般には液水とはその場の水分が飽和水蒸気圧を超えることで生じるが、多孔質材料中では材料の吸着力・毛管力などの影響により飽和水蒸気圧以下で液水が保持されることがある。  
（松本衛：新建築学大系10によれば多孔質材料中では水分は  
- 1.固体実質部の結晶内部や空隙の界面に存在する水分（収着および吸着水）  
- 2.ゲル構造の固体内にオスモティック（浸透圧力による）に保持される水分   
- 3.固体内の毛細管に、毛細管力で保持される水分（毛管水）  
- 4.大きな毛細管、空隙に存在する重力水   
- 5.空隙内に存在する水蒸気。  
に大別される。）  

ここで平衡状態における水分の状態量と、そのときの含水率の関係式を一般に建築などの分野では平衡含水率、土壌やその他の分野では水分特性曲線や水分保持曲線、吸着等温線などと呼称する。  
この関係式は多孔質材料ごとに固有の値を示すため、デシケータ法、チャンバー法、加圧板法など様々な方法によって測定を行う。  

一方で液水の状態量は以下の3つの方法で表現が可能である。 

- $\phi$ 含水率, %  
- $\mu$ 水分化学ポテンシャル, J kg<sup>-1</sup>  
- $P_l$ 溶液の圧力, Pa  
    
一般に水分化学ポテンシャルと溶液の圧力は同一の状態量を表しており、エネルギーであるか圧力であるかの違いである。  
それぞれの物理的意味は以下のように表される。  

### $\phi$ ：含水率  
空間1m<sup>3</sup>あたりに含まれる液水の割合を百分率で表したものを含水率と呼ぶ。  
絶対湿度同様、液水が蒸発または移動しない限り、温度が変わっても変化しない量である。  

### $\mu$：水分化学ポテンシャル  
化学ポテンシャルはJ.W.Gibbsにより提唱された状態変数の一つであり、1molあたりのギブスの自由エネルギーと定義される。
（※ただし1kg当たりのエネルギー量として定義されることが多い。）  
液水の化学ポテンシャルは水蒸気の化学ポテンシャルと基本的な考え方は同じであるが、塩や高分子などを含むことで水の濃度が減少した場合にはその限りではなくなる。（いわゆる蒸気圧降下）  
上述のように、化学ポテンシャルと含水率には吸着力によってある特定の関係が成り立つが、吸湿時と放湿時にはこの関係性が異なっており、この現象のことを吸放湿履歴（単に履歴現象ということも）と呼ぶ。ただし履歴現象はメカニズムおよび測定が難しく、解析には含めないことが多い。  
以上から、液水の水分化学ポテンシャルは平衡含水率関係$f(\phi)$を用いて以下のように表される。  

$\mu = f(\phi)$

ただし、一般に材料の微小部分においては局所平衡が成り立ち、水蒸気と液水の状態量（化学ポテンシャル）が平衡すると仮定することで、水蒸気の化学ポテンシャル同様以下の式で表することができる。

$\mu = R_v T log(RH)$ [J/kg] 

Rvは水蒸気の気体定数である。

### $P_l$：液水の圧力  
土壌の分野では液水の状態量を圧力として定義することが多い。  
一般にこの圧力は水分化学ポテンシャルと等価であり、水の密度$\rho_w$を用いることで以下のように表される。  

$P_l = \rho_w \mu$
